In [1]:
!pip install groq python-dotenv faiss-cpu scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 184.1 kB/s eta 0:00:001m170.1 kB/s eta 0:00:01


In [1]:
import faiss
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample documents (small knowledge base)
documents = [
    "Python is a programming language.",
    "The capital of France is Paris.",
    "Azure is a cloud computing platform by Microsoft.",
    "OpenAI provides powerful language models for various tasks.",
    "The Great Wall of China is a historic landmark."
]

# Convert documents into TF-IDF vectors
vectorizer = TfidfVectorizer()
document_vectors = vectorizer.fit_transform(documents).toarray()

# Index the document vectors using FAISS (for fast retrieval)
index = faiss.IndexFlatL2(document_vectors.shape[1])
index.add(np.array(document_vectors).astype(np.float32))

# Function to retrieve the most relevant document
def retrieve(query, top_n=1):
    query_vector = vectorizer.transform([query]).toarray()
    distances, indices = index.search(query_vector.astype(np.float32), top_n)
    return [documents[i] for i in indices[0]]


In [4]:
import os
from dotenv import load_dotenv
from groq import Groq

# Set up Groq credentials
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
model = "llama-3.3-70b-versatile"

client = Groq(api_key=api_key)


In [5]:
def rag_pipeline(query):
    # Step 1: Retrieve the most relevant document
    retrieved_docs = retrieve(query, top_n=1)
    context = " ".join(retrieved_docs)
    print("Retrieved Document:", context)

    # Step 2: Use Groq to generate an answer using the retrieved document
    prompt = f"Based on the following context, answer the question: {query}\n\nContext: {context}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps people find information.  Only provide answer based on the context that is provided"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=100,
        temperature=0.7
    )

    return response.choices[0].message.content.strip()


In [5]:
response = rag_pipeline("What is the capital of France?")
print("Final Response:", response)


Retrieved Document: The capital of France is Paris.
Final Response: The capital of France is Paris.
